In [ ]:
from utils.helpers import set_api_keys_env
set_api_keys_env()

In [ ]:
from IPython.core.display import display_html

In [ ]:
import os
from tavily import TavilyClient


# connect
client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

In [ ]:
# run search
result = client.search("What is in Nvidia's new Blackwell GPU?",
                       include_answer=True)

# print the answer
result["answer"]


## Regular search

In [ ]:
# choose location (try to change to your own city!)

city = "San Francisco"

query = f"""
    what is the current weather in {city}?
    Should I travel there today?
    "weather.com"
"""

In [ ]:
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import re

ddg = DDGS()

def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        print(f"returning previous results due to exception reaching ddg. {e}")
        results = [ # cover case where DDG rate limits due to high deeplearning.ai volume
            "https://weather.com/weather/today/l/San+Francisco+CA+USCA0987:1:US",
            "https://weather.com/weather/hourbyhour/l/USCA0987:1:US",
            "https://weather.com/weather/hourbyhour/l/San+Jose+CA?canonicalCityId=7b5a3aba31048cd67bd2f5b6519c54be",
            "https://weather.com/weather/tenday/l/Houston+TX?canonicalCityId=7e34fb5e40b0e26a49f097ff4432d602",
            "https://weather.com/weather/today/l/USCA0987:1:US",
            "https://weather.com/weather/tenday/l/Miami+FL?canonicalCityId=146962bd4c383709552626d733ca604b"
        ]
        return results  

ddg_results = search(query)
for i in ddg_results:
    print(i)

In [ ]:
def scrape_weather_info(url):
    """Scrape content from the given URL"""
    if not url:
        return "Weather information could not be found."
    
    # fetch data
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Failed to retrieve the webpage."

    # parse result
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup


In [ ]:
# use DuckDuckGo to find websites and take the first result
url = ddg_results[0]

# scrape first wesbsite
soup = scrape_weather_info(url)

print(f"Website: {url}\n\n")
print(str(soup.body)[:50000]) # limit long outputs

In [ ]:
# extract text
weather_data = []
for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
    text = tag.get_text(" ", strip=True)
    weather_data.append(text)

# combine all elements into a single string
weather_data = "\n".join(weather_data)

# remove all spaces from the combined text
weather_data = re.sub(r'\s+', ' ', weather_data)
    
print(f"Website: {url}\n\n")
print(weather_data)

In [ ]:
# run search
result = client.search(query, max_results=1)

# print first result
data = result["results"][0]["content"]

print(data)

In [ ]:
import json
from pygments import highlight, lexers, formatters

# parse JSON
parsed_json = json.loads(data.replace("'", '"'))

# pretty print JSON with syntax highlighting
formatted_json = json.dumps(parsed_json, indent=4)
colorful_json = highlight(formatted_json,
                          lexers.JsonLexer(),
                          formatters.TerminalFormatter())

print(colorful_json)
